In [9]:
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from kbland import KB_crawling
from selenium.webdriver.chrome.options import Options
from kakaoMap import KakaoMap


### 초기 데이터 작업

In [2]:
df = pd.read_csv('../data/gangnam_22to23.csv')
df

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,...,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명
0,2023,11680,강남구,10100,역삼동,1.0,대지,774.0,34.0,리치웰빌라트2차,...,60000,47.42,29.78,6.0,NaN,NaN,2017.0,연립다세대,중개거래,서울 강남구
1,2023,11680,강남구,11800,도곡동,1.0,대지,961.0,0.0,현대아파트1동~3동,...,138600,84.96,0.00,7.0,NaN,NaN,1994.0,아파트,직거래,NaN
2,2023,11680,강남구,10700,신사동,1.0,대지,555.0,34.0,(555-34),...,19500,27.01,19.42,2.0,NaN,NaN,2011.0,연립다세대,직거래,NaN
3,2023,11680,강남구,10100,역삼동,1.0,대지,823.0,26.0,강남역두산위브센티움,...,21500,28.71,39.30,5.0,NaN,NaN,2004.0,오피스텔,중개거래,서울 강남구
4,2023,11680,강남구,11800,도곡동,1.0,대지,953.0,1.0,SK허브프리모,...,41000,33.80,0.00,17.0,NaN,NaN,2006.0,아파트,직거래,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12799,2022,11680,강남구,10100,역삼동,1.0,대지,837.0,18.0,강남역 서희스타힐스 오피스텔,...,18000,23.58,36.07,6.0,NaN,NaN,2012.0,오피스텔,중개거래,서울 강남구
12800,2022,11680,강남구,10100,역삼동,1.0,대지,837.0,18.0,강남역 서희스타힐스 오피스텔,...,25500,31.50,48.18,12.0,NaN,NaN,2012.0,오피스텔,중개거래,서울 강남구
12801,2022,11680,강남구,10100,역삼동,1.0,대지,837.0,18.0,강남역 서희스타힐스 오피스텔,...,25500,31.50,48.18,12.0,NaN,NaN,2012.0,오피스텔,중개거래,서울 강남구
12802,2022,11680,강남구,10100,역삼동,1.0,대지,837.0,18.0,강남역 서희스타힐스 오피스텔,...,25500,31.50,48.18,12.0,NaN,NaN,2012.0,오피스텔,중개거래,서울 강남구


In [3]:
condition = df['건물용도'] == '아파트'
df['건물면적(㎡)']
APT_df = df[condition]
APT_df

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,...,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명
1,2023,11680,강남구,11800,도곡동,1.0,대지,961.0,0.0,현대아파트1동~3동,...,138600,84.96,0.0,7.0,NaN,NaN,1994.0,아파트,직거래,NaN
4,2023,11680,강남구,11800,도곡동,1.0,대지,953.0,1.0,SK허브프리모,...,41000,33.80,0.0,17.0,NaN,NaN,2006.0,아파트,직거래,NaN
6,2023,11680,강남구,10300,개포동,1.0,대지,1280.0,0.0,래미안블레스티지,...,275000,84.94,0.0,3.0,NaN,NaN,2019.0,아파트,중개거래,"경기 부천시, 서울 강남구"
8,2023,11680,강남구,11800,도곡동,1.0,대지,869.0,0.0,삼익,...,220000,153.71,0.0,3.0,NaN,NaN,1983.0,아파트,중개거래,서울 강남구
9,2023,11680,강남구,10500,삼성동,1.0,대지,87.0,0.0,아이파크,...,590000,156.86,0.0,29.0,NaN,NaN,2004.0,아파트,중개거래,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12790,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,76000,80.39,0.0,5.0,NaN,NaN,2002.0,아파트,중개거래,서울 강남구
12793,2022,11680,강남구,10100,역삼동,1.0,대지,754.0,0.0,개나리래미안,...,325000,129.80,0.0,14.0,NaN,NaN,2006.0,아파트,중개거래,서울 강남구
12795,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,76000,80.39,0.0,5.0,NaN,NaN,2002.0,아파트,중개거래,서울 강남구
12796,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,76000,80.39,0.0,5.0,NaN,NaN,2002.0,아파트,중개거래,서울 강남구


In [4]:
APT_df['방'] = np.nan
APT_df['욕실'] = np.nan
APT_df['주차'] = np.nan
APT_df['초등학교'] = np.nan
APT_df

/var/folders/yl/ctrp4vjd3zn407qhh_nft51h0000gn/T/ipykernel_35169/4049425061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  APT_df['방'] = np.nan
/var/folders/yl/ctrp4vjd3zn407qhh_nft51h0000gn/T/ipykernel_35169/4049425061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  APT_df['욕실'] = np.nan
/var/folders/yl/ctrp4vjd3zn407qhh_nft51h0000gn/T/ipykernel_35169/4049425061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,...,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,방,욕실,주차,초등학교
1,2023,11680,강남구,11800,도곡동,1.0,대지,961.0,0.0,현대아파트1동~3동,...,NaN,NaN,1994.0,아파트,직거래,NaN,NaN,NaN,NaN,NaN
4,2023,11680,강남구,11800,도곡동,1.0,대지,953.0,1.0,SK허브프리모,...,NaN,NaN,2006.0,아파트,직거래,NaN,NaN,NaN,NaN,NaN
6,2023,11680,강남구,10300,개포동,1.0,대지,1280.0,0.0,래미안블레스티지,...,NaN,NaN,2019.0,아파트,중개거래,"경기 부천시, 서울 강남구",NaN,NaN,NaN,NaN
8,2023,11680,강남구,11800,도곡동,1.0,대지,869.0,0.0,삼익,...,NaN,NaN,1983.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
9,2023,11680,강남구,10500,삼성동,1.0,대지,87.0,0.0,아이파크,...,NaN,NaN,2004.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12790,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,NaN,NaN,2002.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
12793,2022,11680,강남구,10100,역삼동,1.0,대지,754.0,0.0,개나리래미안,...,NaN,NaN,2006.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
12795,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,NaN,NaN,2002.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
12796,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,NaN,NaN,2002.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN


### 크롤링코드

In [8]:
APT_df = pd.read_csv('../data/gangnam_plus.csv')
APT_df

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,...,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,방,욕실,주차,초등학교
0,2023,11680,강남구,11800,도곡동,1.0,대지,961.0,0.0,현대아파트1동~3동,...,NaN,NaN,1994.0,아파트,직거래,NaN,3.0,2.0,1.50,358
1,2023,11680,강남구,11800,도곡동,1.0,대지,953.0,1.0,SK허브프리모,...,NaN,NaN,2006.0,아파트,직거래,NaN,1.0,1.0,1.48,278
2,2023,11680,강남구,10300,개포동,1.0,대지,1280.0,0.0,래미안블레스티지,...,NaN,NaN,2019.0,아파트,중개거래,"경기 부천시, 서울 강남구",3.0,2.0,1.61,770
3,2023,11680,강남구,11800,도곡동,1.0,대지,869.0,0.0,삼익,...,NaN,NaN,1983.0,아파트,중개거래,서울 강남구,5.0,2.0,1.10,672
4,2023,11680,강남구,10500,삼성동,1.0,대지,87.0,0.0,아이파크,...,NaN,NaN,2004.0,아파트,중개거래,서울 강남구,3.0,2.0,2.79,294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3906,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,NaN,NaN,2002.0,아파트,중개거래,서울 강남구,2.0,1.0,0.93,976
3907,2022,11680,강남구,10100,역삼동,1.0,대지,754.0,0.0,개나리래미안,...,NaN,NaN,2006.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
3908,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,NaN,NaN,2002.0,아파트,중개거래,서울 강남구,2.0,1.0,0.93,976
3909,2022,11680,강남구,10100,역삼동,1.0,대지,783.0,28.0,갤러리아트,...,NaN,NaN,2002.0,아파트,중개거래,서울 강남구,2.0,1.0,0.93,976


In [32]:
APT_df[APT_df['방'].isna()]

,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,...,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명,방,욕실,주차,초등학교
49,2023,11680,강남구,10500,삼성동,1.0,대지,189.0,0.0,래미안라클래시,...,NaN,NaN,2021.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
76,2023,11680,강남구,10100,역삼동,1.0,대지,763.0,16.0,래미안그레이튼,...,NaN,NaN,2009.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
87,2023,11680,강남구,10600,대치동,1.0,대지,1027.0,0.0,래미안대치팰리스,...,NaN,NaN,2015.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
97,2023,11680,강남구,10600,대치동,1.0,대지,1027.0,0.0,래미안대치팰리스,...,NaN,NaN,2015.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
155,2023,11680,강남구,10600,대치동,1.0,대지,1027.0,0.0,래미안대치팰리스,...,NaN,NaN,2015.0,아파트,중개거래,"서울 강남구, 서울 송파구",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3761,2022,11680,강남구,10500,삼성동,1.0,대지,189.0,0.0,래미안라클래시,...,NaN,NaN,2021.0,아파트,직거래,NaN,NaN,NaN,NaN,NaN
3844,2022,11680,강남구,10300,개포동,1.0,대지,12.0,0.0,성원대치2단지아파트,...,NaN,NaN,1992.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
3845,2022,11680,강남구,10300,개포동,1.0,대지,12.0,0.0,성원대치2단지아파트,...,NaN,NaN,1992.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN
3858,2022,11680,강남구,10300,개포동,1.0,대지,12.0,0.0,성원대치2단지아파트,...,NaN,NaN,1992.0,아파트,중개거래,서울 강남구,NaN,NaN,NaN,NaN


In [10]:
def get_search_query(자치구명, 법정동명, 본번, 부번):
    
    query_without_addNo = f"{자치구명} {법정동명} {int(본번)}"
    query_with_addNo = f"{query_without_addNo}-{int(부번)}"
    
    if 부번 == 0 : return query_without_addNo
    else: return query_with_addNo

In [23]:
option = Options()
option.add_argument('headless')
option.add_argument('window-size=1920x1080')
option.add_argument('disable-gpu')
option.add_argument('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')
option.add_argument('lang=ko_KR')

driver_path = '/opt/homebrew/bin/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path, options=option) # driver 실행

kbland_url = 'https://kbland.kr/map' # kb 부동산 url
driver.get(kbland_url) # kb부동산 페이지 열기

for index, row in APT_df[APT_df['방'].isna()].iterrows():

  building_name = row['건물명']
  search_query = get_search_query(row['자치구명'], row['법정동명'], row['본번'], row['부번'])
  square_meter = int(row['건물면적(㎡)'])
  
  kb = KB_crawling(driver, building_name, search_query, square_meter)

  kb.activate_search_box()
  
  try:
    kb.search_apartment()
    
    kb.click_to_check_detail()

    APT_df.loc[index,'방'], APT_df.loc[index,'욕실'] = kb.get_room_info()
    APT_df.loc[index,'주차'] = kb.get_parking_info()
    APT_df.loc[index,'초등학교'] = kb.get_primary_school_info()

  except:
    pass

  kb.back_to_home()
  
    

In [34]:
kakao = KakaoMap()

for index, row in APT_df.iterrows():
  building_address = get_search_query(row['자치구명'], row['법정동명'], row['본번'], row['부번'])
  try:
    APT_df.loc[index,'가까운 역'], APT_df.loc[index,'역까지의 거리'] = kakao.get_nearest_station_distance(building_address)
  except:
    pass

주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.
주소를 찾을 수 없습니다.


In [35]:
# file 내보내기
file_path = './data/gangnam_plus.csv'
APT_df.to_csv(file_path,index=False)

### 현재 최신 코드

In [17]:
driver_path = '/opt/homebrew/bin/chromedriver'

# driver 실행
driver = webdriver.Chrome(executable_path=driver_path)

# kb 부동산 url
kbland_url = 'https://kbland.kr/map'

#kb부동산 페이지 열기
driver.get(kbland_url)


# driver waits fot maximum 2sec to be appeared
wait = WebDriverWait(driver, 2)


for index, row in APT_df[APT_df['방'].isna()].iterrows():

    # if type(row['방']) == str:
    #     continue

    
    # click to find search box : 검색 상자 활성화
    find_search_box = driver.find_element_by_class_name('homeSerchBox')
    try:
        driver.execute_script("arguments[0].click();", find_search_box)
        # find_search_box.click()

    except ElementClickInterceptedException:
        # 한번 더 클릭하여 팝업창 없애기
        find_search_box.click()
    
    # 검색할 쿼리
    building_name = row['건물명']
    search_query = get_search_query(row['자치구명'], row['법정동명'], row['본번'], row['부번'])
    square_meter = int(row['건물면적(㎡)'])
    
    try:
        search_box = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="text"]:nth-child(1)')))
        search_box.send_keys(search_query)
        search_box.send_keys(Keys.RETURN)

        try:
            wrapper_elements = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='list-search-poi']")))
            # print("Target element exists. Proceed with next code.")
            item_elements = driver.find_elements(By.XPATH, "//span[@class='text']")
            item_text = [item.get_attribute('innerText') for item in item_elements]

            opimal_index = most_similar_string_index(building_name, item_text)
            item_elements[opimal_index].click()

            span_name_value = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'nameValue')))
            span_name_value.click()

            type_elements = wait.until(EC.presence_of_element_located((By.XPATH,f'//span[em and contains(text(), {square_meter})]')))
            next_sibling = type_elements.find_element_by_xpath("following-sibling::*[1]")
            room_text = next_sibling.get_attribute('innerText')

            # 정규표현식을 사용하여 문자 추출
            pattern1 = r'\n(.+?)개'
            pattern2 = r'개/(.+?)개'
            room = re.search(pattern1, room_text).group(1)
            bathroom = re.search(pattern2, room_text).group(1)

            APT_df.loc[index,'방'] = room
            APT_df.loc[index,'욕실'] = bathroom

            # 평균 주차 대 수 찾기
            parking_element = driver.find_element(By.XPATH, '//span[contains(text(), "주차")]')
            parking_text = parking_element.get_attribute('innerText')

            pattern3 = r'주차 (.+?)대'
            parking_lot = re.search(pattern3, parking_text).group(1)

            APT_df.loc[index,'주차'] = parking_lot

            # 배정된 초등학교의 거리 찾기
            primary_school_element = driver.find_element(By.XPATH, '//em[@class="textellipsis" and contains(text(), "초등학교")]')
            grandparent_elememt = primary_school_element.find_element_by_xpath("../..")
            sibling_grandparent = grandparent_elememt.find_element_by_xpath("following-sibling::*[1]")
            school_distance_element = sibling_grandparent.find_element(By.XPATH, '//em[contains(text(), "m")]')

            # 텍스트 추출
            school_distance_text = school_distance_element.get_attribute('innerText')
            pattern4 = r'(.+?)m'
            school_distance = re.search(pattern4, school_distance_text).group(1)
            APT_df.loc[index,'초등학교'] = school_distance
            

        except TimeoutException:
            span_name_value = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'nameValue')))
            span_name_value.click()

            type_elements = wait.until(EC.presence_of_element_located((By.XPATH,f'//span[em and contains(text(), {square_meter})]')))
            next_sibling = type_elements.find_element_by_xpath("following-sibling::*[1]")
            room_text = next_sibling.get_attribute('innerText')

            # 정규표현식을 사용하여 문자 추출
            pattern1 = r'\n(.+?)개'
            pattern2 = r'개/(.+?)개'
            room = re.search(pattern1, room_text).group(1)
            bathroom = re.search(pattern2, room_text).group(1)

            APT_df.loc[index,'방'] = room
            APT_df.loc[index,'욕실'] = bathroom

            # 평균 주차 대 수 찾기
            parking_element = driver.find_element(By.XPATH, '//span[contains(text(), "주차")]')
            parking_text = parking_element.get_attribute('innerText')

            pattern3 = r'주차 (.+?)대'
            parking_lot = re.search(pattern3, parking_text).group(1)

            APT_df.loc[index,'주차'] = parking_lot

            # 배정된 초등학교의 거리 찾기
            primary_school_element = driver.find_element(By.XPATH, '//em[@class="textellipsis" and contains(text(), "초등학교")]')
            grandparent_elememt = primary_school_element.find_element_by_xpath("../..")
            sibling_grandparent = grandparent_elememt.find_element_by_xpath("following-sibling::*[1]")
            school_distance_element = sibling_grandparent.find_element(By.XPATH, '//em[contains(text(), "m")]')

            # 텍스트 추출
            school_distance_text = school_distance_element.get_attribute('innerText')
            pattern4 = r'(.+?)m'
            school_distance = re.search(pattern4, school_distance_text).group(1)
            APT_df.loc[index,'초등학교'] = school_distance

        

    except:
        pass
    
    
    home_btn = driver.find_element(By.XPATH, "//button[@class='btn btn-nav home' and text()=' 홈']")
    driver.execute_script("arguments[0].click();", home_btn)
    

#driver.quit()

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=115.0.5790.170)
